In [ ]:
%config Completer.use_jedi = False
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
import pandas as pd
import re
import os
import jellyfish
from collections import Counter
import time

In [ ]:
# ["python", "data science", "copyright", "comptability", "design", "excel, javascript", "adobe premiere", "photoshop", " c++", "guitare", "math", "piano", "violon", "java"]
subject = 'photoshop'

try: os.mkdir('./data/' + subject)
except FileExistsError: pass

In [ ]:
with open(f'./data/{subject}/cards.csv', 'a') as f:
    if f.tell() == 0:
        f.write("name;price;nb_comments;note;profil_link;description")
    for p in range(10):
        options = Options()
        options.headless = True
        driver = webdriver.Firefox(options=options)
        driver.get(f"https://fr.fiverr.com/search/gigs?query={subject}&source=pagination&search_in=everywhere&search-autocomplete-original-term={subject}&page={p}")

        if 'block.fiverr.com' in driver.current_url or '/404' in driver.current_url:
            print('blocked')
            raise Exception

        elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'gig-card-layout')]")
        for i, element in enumerate(elements):
            try: 
                name = element.find_element(By.XPATH, ".//div[contains(@class, 'seller-name')]/a").text
                profil_link = element.find_element(By.XPATH, ".//div[contains(@class, 'seller-name')]/a").get_attribute('href')
                description = element.find_element(By.XPATH, ".//h3/a").text.replace(';', ',')
                ratingText = element.find_element(By.XPATH, ".//span[contains(@class, 'gig-rating')]").text
                groups = re.match(r'(\d)(?:,)(\d)(?:\()(\d+)(?:\))', ratingText).groups()
                note = int(groups[0])+int(groups[1])/10
                price = int(element.find_element(By.XPATH, ".//a[contains(@class, 'price')]/span").text[:-2])/100
                
            except Exception as e:
                print(name)
                    
            f.write(f"{name};{price};{int(groups[2])};{note};{profil_link};{description}\n")
            
    del driver, options, elements, name, profil_link, description, ratingText, groups, note, price, i, p, f

In [ ]:
cards = pd.read_csv(f'./data/{subject}/cards.csv', sep=';')

In [ ]:
with open(f'./data/{subject}/profils.csv', 'a') as f:
    if f.tell() == 0:
        f.write(f"name;location;created_at;response_time;last_order:languages;linked_acc;skills;education;description\n")

    for id in range(30):
        options = Options()
        options.headless = True
        driver = webdriver.Firefox(options=options)
        driver.get(cards.loc[id]['profil_link'])
        # The website can block from request or the page doesn't exist or the profil link doesn't exist so it returns to the main page
        if 'block.fiverr.com' in driver.current_url or '/404' in driver.current_url or len(driver.current_url) < 25:
            raise Exception

        name = cards.loc[id]['name']
        stats = [i.text.strip() for i in driver.find_elements(By.XPATH, "//ul[contains(@class, 'user-stats')]/li/*[self::b or self::strong]")]
        location = stats[0]
        created_at = stats[1]
        response_time = stats[2]
        last_order = stats[3]
        languages = [f"{i.text[i.text.find('(')+1:i.text.find(')')]}-{i.text.split('-')[-1].strip()}" for i in driver.find_elements(By.XPATH, "//div[contains(@class, 'languages')]/ul/li")]
        linked_acc = [i.text for i in driver.find_elements(By.XPATH, "//div[contains(@class, 'linked-accounts')]/ul/li/span[@class='text']")]
        skills = [i.text for i in driver.find_elements(By.XPATH, "//div[contains(@class, 'skills')]/ul/li/a")]
        try:
            education = driver.find_element(By.XPATH, "//div[contains(@class, 'education-list')]/ul/li/p").text
        except Exception:
            education = ''
        description = driver.find_element(By.XPATH, "//div[contains(@class, 'description')]/p").text.replace(';', ',')
        
        f.write(f"{name};{location};{created_at};{response_time};{last_order};{'|'.join(languages)};{'|'.join(linked_acc)};{'|'.join(skills)};{education};{description}\n")
        
        del driver, options, name, stats, location, created_at, response_time, last_order, languages, linked_acc, skills, education, description
        
    del id, f

In [ ]:
profils = pd.read_csv(f'./data/{subject}/profils.csv', sep=';')

In [ ]:
def get_common_skills(l: list, nb: int):
    skills = Counter({})
    for row in l:
        temp = row.split('|')
        for i in temp:
            skills[i] += 1
    return skills.most_common(nb)

In [ ]:
get_common_skills(list(profils['skills']), 5)